In [2]:
from src.preprocessing.preprocess import *

process_org_data('oakridge', ['Time (second)', 'Time', 'TC1 (°C)', '°C', '[C]'])

In [8]:
df = pd.read_excel('data/raw/oakridge/excel/LFP_15Ah_60SOC_cell1_MAX.xlsx')
expected_substrings = ['Time (second)', 'Time', 'TC1 (°C)', '°C', '[C]']

# cols = []
# for col in df.columns:
#     print(f'{col}: ')
#     for sub in expected_substrings:
#         print(f'\t{sub} in {col}? {(sub in str(col))}')

df[[col for col in df.columns if any(sub in str(col) for sub in expected_substrings)]]

,Time,Function 2 [C]
0,0.000,22.97453
1,0.045,22.97453
2,0.091,22.95643
3,0.136,22.95643
4,0.181,22.93832
...,...,...
5570,539.727,NaN
5571,539.827,NaN
5572,539.928,NaN
5573,540.027,NaN


In [6]:
ORG_RAW_DATA = {
    'oakridge': 'data/raw/oakridge/excel',
}

orgname = 'oakridge'
input_dir = ORG_RAW_DATA.get(orgname,f'{config.DATA_DIR}raw/{orgname}/')
output_dir = f'{config.DATA_DIR}preprocessed/{orgname}/'

input_dir, output_dir

('data/raw/oakridge/excel', 'data/preprocessed/oakridge/')

In [43]:
files = ['OE-LCO-6500mAh-20SOC',
 'SNL_NMC-LMO_Graphite_26Ah_90SOC_b',
 'LCO_4000mAh-10SOC_cell2_MAX',
 'LFP_15Ah_40SOC_cell1_MAX',
 'Soteria-control-30SOC-cell1',
 'OE-LFP10Ah-80SOC-Cell15-origina',
 'OE-10Ahr-NMC-70SOC-cell6',
 '500mAh3-20S0C',
 '2000mAh1-40S0C',
 'SNL_LCO_Graphite_6.4Ah_50SOC_a',
 'SNL_LCO_Graphite_6.4Ah_100SOC_b',
 'ChevyVolt-60SOC-6mm-cell2',
 'OE-NMC10Ah-20SOC',
 'ChevyVolt-0SOC-6mm-cell1',
 'Copy of SNL_LMO-LNO_Graphite_33Ah_50SOC_b-copy',]

import re 
file = 'OE-LCO-6500mAh-20SOC'
print(re.split(r'[-_]+', file))

print(re.search(r'(\d+)[S0]OC', file))
print(int(match.group(1)) if (match := re.search(r'(\d+)[S0]OC', file)) else None)

print('num files: ',len(files))
df = pd.read_excel('data/raw/oakridge/main.xlsx', header=None)
df = df[df[0].isin(files)]
df.head(10)

['OE', 'LCO', '6500mAh', '20SOC']
<re.Match object; span=(15, 20), match='20SOC'>
20
num files:  15


,0,1,2,3,4,5,6,7,8,9,10,11,12
14,Soteria-control-30SOC-cell1,NaN,Completed,NaN,"Physical effect (pouch swelling), extended jou...",5190,30,NaN,NaN,NaN,NaN,NaN,NaN
24,OE-LCO-6500mAh-20SOC,NaN,Completed,NaN,"Moderate effect, extended joule heating, local...",6500,20,NaN,NaN,NaN,NaN,NaN,NaN
49,OE-10Ahr-NMC-70SOC-cell6,NaN,Completed,NaN,"Physical effect, rupture of pouch, smoke, gas ...",10000,70,NaN,NaN,NaN,NaN,NaN,NaN
71,500mAh3-20S0C,NaN,Completed,NaN,"No effect, local heating, internal discharge – 2",500,20,NaN,NaN,NaN,NaN,NaN,NaN
77,2000mAh1-40S0C,NaN,Completed,NaN,"No effect, local heating, internal discharge – 2",2000,40,NaN,NaN,NaN,NaN,NaN,NaN
104,OE-NMC10Ah-20SOC,NaN,Completed,NaN,"Moderate effect, extended joule heating, local...",10000,20,NaN,NaN,NaN,NaN,NaN,NaN
118,ChevyVolt-0SOC-6mm-cell1,NaN,Completed,NaN,NaN,26000,0,NaN,NaN,NaN,NaN,NaN,NaN
127,ChevyVolt-60SOC-6mm-cell2,NaN,Completed,NaN,NaN,26000,60,NaN,NaN,NaN,NaN,NaN,NaN
152,LCO_4000mAh-10SOC_cell2_MAX,NaN,Completed,NaN,NaN,4000,10,NaN,NaN,NaN,NaN,NaN,NaN
168,LFP_15Ah_40SOC_cell1_MAX,NaN,Completed,NaN,NaN,15000,40,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.loc[df[0] == 'Soteria-control-30SOC-cell1']

,0,1,2,3,4,5,6,7,8,9,10,11,12
14,Soteria-control-30SOC-cell1,NaN,Completed,NaN,"Physical effect (pouch swelling), extended jou...",5190,30,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# rows = df[(df[6] < 100) & (df[6] > 30)]
# rows[0].tolist()

folder_path = 'data/preprocessed/oakridge/'
os.listdir(folder_path)
processed_files = [os.path.splitext(f)[0] for f in os.listdir(folder_path)]
print('num files = ', len(processed_files))

folder_path = 'data/raw/oakridge/excel/'
os.listdir(folder_path)
files = [os.path.splitext(f)[0] for f in os.listdir(folder_path)]
print('num files = ', len(files))
set(files) - set(processed_files)

num files =  253
num files =  253


set()

In [11]:
for file_path in glob(os.path.join(input_dir, '*')):
    df = pd.read_excel(file_path)
    print(file_path.split('/')[-1])
    print(df.columns.tolist())
    print()

OE-LCO-6500mAh-20SOC.xlsx
['Time (second)', 'Load (lb)', 'Voltage (V)', 'Unnamed: 3', 'Unnamed: 4', 'Time (sec)', 'Penetrator Force (N)', 'Cell Voltage (V)', 'Displacement (mm)', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Time (sec) ', 'TC1 (°C)', 'TC2 (°C)', 'TC3 (°C)', 'TC4 (°C)']

SNL_NMC-LMO_Graphite_26Ah_90SOC_b.xlsx
['Test Time [s]', 'Displacement [mm]', 'Penetrator Force [mm]', 'vCell [V]', 'tAmbient [C]', 'TC1 near positive terminal [C]', 'TC2 near negative terminal [C]', 'TC3 bottom - bottom [C]', 'TC4 bottom - top [C]', 'TC5 above punch [C]', 'TC6 below punch [C]']

LCO_4000mAh-10SOC_cell2_MAX.xlsx
['Time', 'Load (lb)', 'Voltage (V)', 'Unnamed: 3', 'reltime', 'MAX [C]']

LFP_15Ah_40SOC_cell1_MAX.xlsx
['Time', 'Load (lb)', 'Voltage (V)', 'Unnamed: 3', 'reltime', 'Function 3 [C]']

Soteria-control-30SOC-cell1.xlsx
['Column1', 'Column2', 'Column3']

OE-LFP10Ah-80SOC-Cell15-

In [5]:
expected_substrings = ['Time (sec)', 'TC1 (°C)', '[C]', 'Time']
df = pd.read_excel('data/raw/oakridge/excel/SNL_NMC-LMO_Graphite_26Ah_50SOC_b.xlsx')
df = df[[col for col in df.columns if any(sub in col for sub in expected_substrings)]]
df.head()

,Test Time [s],tAmbient [C],TC1 near positive terminal [C],TC2 near negative terminal [C],TC3 bottom - bottom [C],TC4 bottom - top [C],TC5 above punch [C],TC6 below punch [C]
0,1.06,16.7,16.0,16.0,16.0,16.2,16.2,16.1
1,2.06,16.7,16.0,16.0,16.0,16.2,16.2,16.1
2,3.06,16.7,16.0,16.0,16.0,16.2,16.2,16.1
3,4.06,16.7,16.0,16.0,16.0,16.2,16.2,16.1
4,5.06,16.7,16.0,16.0,16.0,16.2,16.2,16.1
